In [370]:
import random as rand
import numpy as np

In [4]:
def coin_flipper(flips):
    num_heads = 0
    for i in range(flips):
        if rand.randint(0, 1) == 1:
            num_heads += 1
    return num_heads

def flip_multiple_coins(coins, flips):
    result = []
    for i in range(coins):
        num_heads = coin_flipper(flips)
        result.append(num_heads)
    return result

def find_desired_coins(coins, flips):
    outcome = flip_multiple_coins(coins, flips)
    least = outcome[0]
    for i in range(len(outcome)):
        if outcome[i] < least:
            least = outcome[i]
    return(outcome[0], rand.choice(outcome), least)

In [11]:
v1 = 0
vrand = 0
vmin = 0
for i in range(100000):
    final = find_desired_coins(1000, 10)
    v1 += final[0]
    vrand += final[1]
    vmin += final[2]
(v1 / 1000000, vrand / 1000000, vmin / 1000000)

(0.500445, 0.500238, 0.037549)

In [387]:
def point_generator(num):
    points = []
    for i in range(num):
        x_coord = rand.uniform(-1, 1)
        y_coord = rand.uniform(-1, 1)
        point = [1, x_coord, y_coord]
        points.append(point)
    return points

def line_maker():
    point1 = (rand.uniform(-1, 1), rand.uniform(-1, 1))
    point2 = (rand.uniform(-1, 1), rand.uniform(-1, 1))
    m = (point2[1] - point1[1]) / (point2[0] - point1[0])
    intercept = -m * point1[0] + point1[1]
    return (m, intercept)

def classifier(points, line):
    label = 0
    training_set = []
    for i in range(len(points)):
        if points[i][2] > points[i][1] * line[0] + line[1]:
            label = 1
        else:
            label = -1
        training_set.append((1, points[i][1], points[i][2], label))
    return training_set

def linear_regression(training_pts):
    y = []
    missclassified = 0
    pts = np.asarray(point_generator(training_pts))
    line = line_maker()
    labeled_pts = classifier(pts, line)
    pseudo_inverse = np.linalg.pinv(pts)
    for i in range(training_pts):
        y.append(labeled_pts[i][3])
    y = np.asarray(y)
    w = np.matmul(pseudo_inverse, y)
    for i in range(len(pts)):
        if np.sign(w.dot(pts[i])) != labeled_pts[i][3]:
            missclassified += 1
    return (w, line, missclassified / len(pts)) 

def linear_regression_nonlinear_classifier(training_pts):
    y = []
    missclassified = 0
    pts = point_generator(training_pts)
    labeled_pts = nonlinear_classifier(pts)
    pseudo_inverse = np.linalg.pinv(pts)
    for i in range(training_pts):
        y.append(labeled_pts[i][3])
    y = np.asarray(y)
    w = np.matmul(pseudo_inverse, y)
    for i in range(len(pts)):
        if np.sign(w.dot(pts[i])) != labeled_pts[i][3]:
            missclassified += 1
    return (w, missclassified / len(pts))   

def nonlinear_transformation_regression(training_pts):
    y = []
    missclassified = 0
    pts = point_generator(training_pts)
    transformed_pts = np.asarray(nonlinear_transformation(pts))
    labeled_pts = nonlinear_classifier(pts)
    pseudo_inverse = np.linalg.pinv(transformed_pts)
    for i in range(training_pts):
        y.append(labeled_pts[i][3])
    y = np.asarray(y)
    w = np.matmul(pseudo_inverse, y)
    for i in range(len(pts)):
        if np.sign(w.dot(transformed_pts[i])) != labeled_pts[i][3]:
            missclassified += 1
    return (w, missclassified / len(pts)) 

def perceptron(num, line, w):
    points = point_generator(num)
    target_function = line
    labeled_set = classifier(points, target_function)
    weight_vector = w
    misclassified = labeled_set
    iterations = 0
    while len(misclassified) > 0:
        iterations += 1
        mpoint = rand.choice(misclassified)
        label = mpoint[3]
        for i in range(3):
            weight_vector[i] += mpoint[i] * label
        misclassified = []
        for i in range(len(labeled_set)):
            if np.sign(weight_vector[0] * labeled_set[i][0] +
                    weight_vector[1] * labeled_set[i][1] +
                    weight_vector[2] * labeled_set[i][2]) != labeled_set[i][3]:
                misclassified.append(labeled_set[i])
    return (weight_vector, iterations, target_function)

In [377]:
lines = []
hypotheses = []
e_in = 0
for i in range(1000):
    x = linear_regression(100)
    hypotheses.append(x[0])
    lines.append(x[1])
    e_in += x[2]
avg_e_in = e_in / 1000
avg_e_in

0.0384800000000001

In [378]:
e_out = 0
for i in range(1000):
    missclassified = 0
    pts = point_generator(1000)
    labeled_pts = classifier(pts, lines[i])
    for j in range(len(pts)):
        if np.sign(hypotheses[i].dot(pts[j])) != labeled_pts[j][3]:
            missclassified += 1
    e_out += (missclassified / 1000)
avg_e_out = e_out / 1000
avg_e_out

0.04830400000000006

In [380]:
iters = 0
for i in range(1000):
    reg = linear_regression(10)
    p = perceptron(10, reg[1], reg[0])
    iters += p[1]
avg_iters = iters / 1000
avg_iters

6.801

In [381]:
def nonlinear_classifier(pts):
    for pt in pts:
        noise = rand.randint(0, 9)
        classification = np.sign(pt[1] ** 2 + pt[2] ** 2 - 0.6)
        if noise != 0:
            pt.append(classification)
        else:
            pt.append(classification * -1)
    return pts

In [382]:
def nonlinear_classifier_no_noise(pts):
    for pt in pts:
        classification = np.sign(pt[1] ** 2 + pt[2] ** 2 - 0.6)
        pt.append(classification)
    return pts

In [388]:
e_in_nonlinear = 0
for i in range(1000):
    pts = nonlinear_function(1000)
    x = linear_regression_nonlinear_classifier(pts)
    e_in_nonlinear += x[2]
avg_e_in_nonlinear = e_in_nonlinear / 1000
avg_e_in_nonlinear

0.0

In [315]:
def nonlinear_transformation(pts):
    transformation = []
    for pt in pts:
        transformation.append([1, pt[1], pt[2], pt[1] * pt[2], pt[1] ** 2, pt[2] ** 2])
    return transformation

In [384]:
w = [0, 0, 0, 0, 0, 0]
for i in range(1000):
    x = nonlinear_transformation_regression(1000)
    for i in range(len(w)):
        w[i] += x[0][i]
for j in range(len(w)):
    w[j] /= 1000
w = np.asarray(w)
w

array([-9.93541556e-01,  5.25467331e-04,  1.61547643e-03,  1.29114920e-03,
        1.56112284e+00,  1.55748937e+00])

In [385]:
adiff = 0
bdiff = 0
cdiff = 0
ddiff = 0
ediff = 0
for i in range(1000):
    x = nonlinear_transformation_regression(1000)
    a = np.asarray([-1,  -0.05,  0.08,  0.13,
        1.5,  1.5])
    b = np.asarray([-1,  -0.05,  0.08,  0.13,
        1.5,  15])
    c = np.asarray([-1,  -0.05,  0.08,  0.13,
        15,  1.5])
    d = np.asarray([-1,  -1.5,  0.08,  0.13,
        0.05,  0.05])
    e = np.asarray([-1,  -0.05,  0.08,  1.5,
        0.15,  0.15])
    pts = point_generator(1000)
    transformed_pts = nonlinear_transformation(pts)
    for j in range(len(pts)):
        if np.sign(a.dot(transformed_pts[j])) == np.sign(x[0].dot(transformed_pts[j])):
            adiff += 1
        if np.sign(b.dot(transformed_pts[j])) == np.sign(x[0].dot(transformed_pts[j])):
            bdiff += 1
        if np.sign(c.dot(transformed_pts[j])) == np.sign(x[0].dot(transformed_pts[j])):
            cdiff += 1
        if np.sign(d.dot(transformed_pts[j])) == np.sign(x[0].dot(transformed_pts[j])):
            ddiff += 1
        if np.sign(e.dot(transformed_pts[j])) == np.sign(x[0].dot(transformed_pts[j])):
            ediff += 1
print((adiff / 1000000, bdiff / 1000000, cdiff / 1000000, ddiff / 1000000, ediff / 1000000))

(0.961905, 0.662485, 0.661054, 0.632023, 0.560939)


In [386]:
e_out_transformation = 0
for i in range(1000):
    missclassified = 0
    pts = point_generator(1000)
    transformed = np.asarray(nonlinear_transformation(pts))
    labeled_pts = nonlinear_classifier(pts)
    for i in range(len(pts)):
        if np.sign(w.dot(transformed[i])) != labeled_pts[i][3]:
            missclassified += 1
    e_out_transformation += (missclassified / 1000)
avg_e_out_transformation = e_out_transformation / 1000
avg_e_out_transformation

0.12399699999999995